# 0. Opdracht Beschrijving

## 0.1. Probleemstelling

Bij onderzoeken naar gewelddadige misdrijven zoals overvallen en schietincidenten is het voor de FBI vaak lastig om snel en betrouwbaar vast te stellen welk type vuurwapen is gebruikt. In veel gevallen is direct fysiek bewijs beperkt of nog niet beschikbaar, terwijl er wel andere vormen van informatie aanwezig zijn, zoals geluidsopnames van schoten, afbeeldingen van wapens of tekstuele beschrijvingen in rapporten en getuigenverklaringen. Het ontbreken van een duidelijke vaststelling van het wapentype kan het opsporingsproces vertragen en de onderbouwing van bewijs in de rechtbank verzwakken. Het doel van ons project is het ontwikkelen van een data engineering pipeline die audio, afbeeldingen en tekst data samenbrengt om het type vuurwapen te voorspellen. Het "model" richt zich op het classificeren van wapens in categorieen zoals pistool, revolver, geweer, shotgun en automatisch wapen. Door kenmerken zoals schiet tempo, kaliber, actie-type en fysieke eigenschappen van het wapen te combineren, kan het systeem de FBI ondersteunen bij het inschatten van het type misdrijf en de ernst ervan. Hiermee wordt het analyseproces versneld en wordt aanvullende objectieve ondersteuning geleverd voor strafzaken.

## 0.2. Zakelijke Inzicht En Literatuur Onderzoek

Vanuit zakelijk perspectief is dit project gericht op het ondersteunen van FBI-rechercheurs bij het herkennen van het type vuurwapen dat is gebruikt bij een misdrijf. Deze informatie kan helpen om een zaak beter te begrijpen en sneller beslissingen te nemen tijdens een onderzoek. Het systeem is bedoeld als een extra informatiebron die wordt gecombineerd met ander bewijsmateriaal.

Het onderzoek binnen dit project richt zich op het vinden en gebruiken van geschikte databronnen voor het automatisch herkennen van vuurwapentypen. Voor audio is een publieke Kaggledataset met vuurwapengeluiden gebruikt, waarmee kenmerken zoals het gemiddeld aantal schoten per tijdseenheid kunnen worden bepaald. Tekstuele informatie is verzameld via webscraping van Guns Fandom, waar beschrijvingen van wapens beschikbaar zijn. Afbeeldingen van vuurwapens zijn opgehaald via de Wikimedia Commons API. De data uit deze bronnen bestaan uit ongestructureerde en semi-gestructureerde formaten en worden verwerkt tot een gestructureerde dataset die geschikt is voor verdere analyse en classificatie.

# 1. Imports En Data Extraction

## 1.1 Packages

In [ ]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
import librosa
import random
import time
import requests
from urllib.parse import unquote

## 1.2 Audio Dataset Van kaggle.com

Voor de audioanalyse wordt een publieke Kaggle dataset met vuurwapengeluiden gebruikt. De dataset bevat 58 verschillende vuurwapens, waarbij per vuurwapen meerdere korte audiofragmenten beschikbaar zijn. Elk audiofragment heeft een vaste lengte van 0,5 seconde en bevat een schot of korte schotreeks. Deze dataset is als eerste gekozen, omdat geschikte audiodata het moeilijkst te vinden was voor dit project. De audio vormt daarom de basis van de verdere dataverzameling. De audiobestanden zijn gestructureerd in mappen met de naam van het vuurwapen, waardoor deze mapnamen gebruikt kunnen worden als metadata. Op basis hiervan is een dataframe opgebouwd waarin het vuurwapenmodel en het bijbehorende audio zijn vastgelegd.

In [ ]:
audio_path = "data/audio/original_dataset/"
audio_files = os.listdir(audio_path)
audio_df = pd.DataFrame({
    'model': [os.path.splitext(f)[0] for f in audio_files],
    'path': [os.path.join(audio_path, f) for f in audio_files]
})
print(audio_df.head(50))

## 1.3 Image API Van wikimedia.org

Voor de beeldanalyse wordt gebruikgemaakt van afbeeldingen die via de Wikimedia Commons API zijn opgehaald. Wikimedia is gekozen omdat het een grote, publieke en vrij toegankelijke verzameling bevat van vuurwapenafbeeldingen, gestructureerd per categorie en voorzien van metadata. De afbeeldingen worden opgehaald op basis van modelnamen van vuurwapens, die eerder zijn verkregen uit de audiodataset. Hierdoor kan dezelfde firearm naam worden gebruikt over meerdere databronnen heen. De Wikimedia API levert semi-gestructureerde data in JSON-formaat. Eerst wordt via een zoekquery gezocht naar relevante afbeelding titels binnen de bestanden. Vervolgens worden per titel de directe image URL’s opgevraagd en lokaal opgeslagen. Alleen bestanden met een geldig image content type worden geaccepteerd, zodat niet relevante resultaten zoals documenten of HTML-pagina’s worden gefilterd. De opgehaalde afbeeldingen worden opgeslagen als JPG-bestanden, met daarnaast een csv-bestand waarin per afbeelding het vuurwapenmodel, de bron URL en het lokale bestandspad zijn vastgelegd. Deze metadata maakt het mogelijk om de afbeeldingen later te koppelen aan andere modaliteiten in het project. De beelden worden in een volgende stap gebruikt voor eenvoudige image analyse, zoals het bepalen van grip_style (eenhandig of tweehandig) en barrel_length (kort of lang). Op deze manier vormt de Wikimedia API een betrouwbare en schaalbare bron voor visuele informatie binnen de multimodale dataset.

In [ ]:
from image_api import ImageAPI
API_URL = "https://commons.wikimedia.org/w/api.php"
HEADERS = {"User-Agent": "Pipline/1.0 (mkakol.index@gmail.com)"}
audio_files = os.listdir(audio_path)
models = [os.path.splitext(f)[0] for f in audio_files]
image_df = ImageAPI(API_URL, HEADERS).fetch_images_for_models(models)

## 1.4 Web Scraping Text Van gunsfandom.com

Voor het bepalen van het wapen type wordt tekstuele informatie verzameld van GunsFandom.com. Deze website is gekozen omdat per vuurwapen een beschrijvende introductietekst beschikbaar is waarin vaak expliciet wordt aangegeven om welk type wapen het gaat, zoals bijvoorbeeld assault rifle, pistol of shotgun. Deze beschrijving vormt de basis voor de targetvariabele weapon_type en is daarmee een belangrijk onderdeel van de classificatie.

Een eerdere aanpak, waarbij direct pagina-URL’s werden opgebouwd op basis van de vuurwapennaam of waarbij vaste pagina elementen werden aangesproken, bleek in de praktijk onvoldoende betrouwbaar. Dit kwam onder andere doordat GunsFandom gebruikmaakt van JavaScript gegenereerde content en doordat vuurwapennamen niet altijd exact overeenkomen met de daadwerkelijke paginanamen op de website. Hierdoor werden pagina’s regelmatig niet gevonden of werd onjuiste inhoud opgehaald. Om dit probleem te verhelpen wordt nu gebruikgemaakt van een zoekgebaseerde web scraping aanpak. Voor elk vuurwapenmodel wordt eerst de interne zoekfunctie van GunsFandom gebruikt om de juiste pagina te vinden. Vervolgens wordt de bijbehorende vuurwapenpagina opgehaald en wordt de eerste betekenisvolle paragraaf uit de hoofdtekst geextraheerd. Deze paragraaf bevat doorgaans een algemene beschrijving van het wapen en vormt daarmee een consistente bron van informatie.

De verkregen tekstdata is ongestructureerd en wordt in eerste instantie ongewijzigd opgeslagen, zodat de originele terminologie van de bron behouden blijft. In een latere fase van feature engineering wordt deze tekst omgezet naar gestandaardiseerde categorieen (zoals pistol, rifle en shotgun) die uiteindelijk worden gebruikt als target voor het "classificatiemodel".

In [ ]:
from text_webscraping import get_fandom_text
url = "https://guns.fandom.com/wiki/AK-47"
paras = get_fandom_text(url)
for i, p in enumerate(paras[:10]):
    print(p)

In [ ]:
from text_webscraping import get_fandom_texts
text_df = get_fandom_texts(audio_df)
pd.set_option("display.max_colwidth", None)
print(text_df)

In [18]:
print(text_df)

                          model  \
0           38sws-dot38-caliber   
1                         ak-12   
2                         ak-47   
3         arctic-warfare-magnum   
4                    beretta-92   
5                     colt-1911   
6                     daewoo-k2   
7                  desert-eagle   
8         double-barrel-shotgun   
9                     dp-27-lmg   
10                       fn-p90   
11                      fn-scar   
12                        glock   
13                    glock-18c   
14    glock-19-9mm-luger-pistol   
15           high-standard-22lr   
16                      hk-g36c   
17                     hk-ump45   
18  hk-usp-compact-40-sw-pistol   
19                       kar98k   
20                 kimber-45acp   
21                 kriss-vector   
22                lorcin-380acp   
23                          m16   
24                      m24-sws   
25                     m249-lmg   
26                           m4   
27                  

# 2. EDA

## 2.1 Audio EDA

We nemen de audio dataset als basis, omdat deze als enige direct gestructureerd is. Elke subfolder is genoemd naar een firearm, wat we gebruiken als identifier om data te koppelen aan afbeeldingen of tekst. Zo hoeven we de namen niet handmatig in te vullen en vormt het een natuurlijke basis voor de pipeline.

De dataset bevat 1450 bestanden, verdeeld over 58 unieke vuurwapens, dus 25 audio’s per wapen. Dit helpt om de variatie in geluid per wapen te bekijken. Het doel is het berekenen van avg_shots_per_sec en kijken naar correlatie met de target weapon_type (pistol, rifle, shotgun).

Elke sample is 0,5 seconde lang, met een sample rate van 22050 Hz. Het exact detecteren van schoten bleek lastig, daarom is een aangepaste functie gebruikt die RMS-energie van de audio berekent om de pieken stabieler te detecteren. Door parameters voor prominente pieken, minimumhoogte en minimale afstand tussen pieken aan te passen, kan de functie best wel betrouwbaar zien of er een of meerdere schoten in een fragment zijn, ook bij korte clips met echo of ruis.

Voor feature engineering trekken we metadata uit de audio: het aantal gedetecteerde pieken voor avg_shots_per_sec en de amplitude voor avg_loudness. Signalen bevatten veel ruis, maar meestal overheerst een duidelijke frequentie, waardoor een uitgebreide Fourier analyse niet nodig is. Gemeten minimum en piekfrequenties liggen tussen 2 en 3844 Hz, wat bruikbaar is voor het beoordelen van het volume en de schot activiteit.

In [ ]:
from audio_dataset import audio_eda
audio_df, counts, dur, srates, peak_freqs = audio_eda("data/audio/original_dataset")
print(audio_df.head())

## 2.2 Image EDA

Inladen van de metadata

In [ ]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os

img_df = pd.read_csv("data/images/image_metadata.csv")
img_df.head()

Controleer of alle beelden goed zijn gedownload

In [ ]:
img_df["exists"] = img_df["path"].apply(lambda p: os.path.exists(p))
img_df

Basis EDA: Afmetingen & bestandsgrootte

In [ ]:
def get_img_stats(path):
    try:
        img = Image.open(path)
        width, height = img.size
        mode = img.mode
        file_size = os.path.getsize(path)
        return width, height, mode, file_size
    except:
        return None, None, None, None

img_df[["width", "height", "mode", "filesize"]] = img_df["path"].apply(
    lambda p: pd.Series(get_img_stats(p))
)

img_df

Visualiseer sample-afbeeldingen

In [ ]:
plt.figure(figsize=(12, 6))
for idx, row in img_df.head(3).iterrows():
    img = Image.open(row["path"])
    plt.subplot(1, 3, idx+1)
    plt.imshow(img)
    plt.title(row["model"])
    plt.axis("off")
plt.show()

Analyse van resoluties

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(img_df["width"], img_df["height"])
plt.xlabel("Width")
plt.ylabel("Height")
plt.title("Image Resolution Distribution")
plt.show()

## 2.3 Text EDA

In [49]:
import re
def clean_weapon_data(df):
    def simple_extract(text):
        if not text or text in ["Null", "Not found"]:
            return "Not found", "Not found"
        text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
        text = re.sub(r'(\d)([A-Z])', r'\1 \2', text)
        # GenAi, Getting NLP right for filtering: https://chatgpt.com/share/69737cb1-59a8-800a-b37a-25b53033de4e
        type_match = re.search(r'weapon\s*type\s*:?\s*([A-Za-z0-9\s\-\(\),\.]+?)(?=\s*caliber|\s*action|\s*length|$)', text, re.IGNORECASE)
        w_type = type_match.group(1).strip() if type_match else "Not found"
        cal_match = re.search(r'caliber\s*:?\s*([\d\.\w\s\-x/]+)', text, re.IGNORECASE)
        caliber = cal_match.group(1).strip() if cal_match else ""
        caliber = re.split(r'\s{2,}|(?=[A-Z][a-z])', caliber)[0].strip()
        return w_type, caliber
    df[['type', 'caliber']] = df['description_text'].apply(lambda x: pd.Series(simple_extract(x)))
    return df[['model', 'type', 'caliber']]
final_df = clean_weapon_data(text_df)
print(final_df.head(15))

                        model  \
0         38sws-dot38-caliber   
1                       ak-12   
2                       ak-47   
3       arctic-warfare-magnum   
4                  beretta-92   
5                   colt-1911   
6                   daewoo-k2   
7                desert-eagle   
8       double-barrel-shotgun   
9                   dp-27-lmg   
10                     fn-p90   
11                    fn-scar   
12                      glock   
13                  glock-18c   
14  glock-19-9mm-luger-pistol   

                                                                           type  \
0                                                                     Not found   
1                                                                 Assault rifle   
2                                                                 Assault rifle   
3                                                                     Not found   
4                                                       

In [42]:
import pandas as pd

def clean_weapon_data(df):
    def simple_extract(text):
        if not text or "Not found" in text:
            return "Not found", "Not found"

        # 1. Split the text into two halves: before "Caliber" and after "Caliber"
        parts = re.split(r'caliber', text, flags=re.IGNORECASE)

        # 2. TYPE is in the first half (after the word "type")
        before_cal = parts[0]
        type_match = re.search(r'type\s*:?\s*(.*)', before_cal, re.IGNORECASE)
        w_type = type_match.group(1).strip() if type_match else "Not found"

        # 3. CALIBER is in the second half
        caliber = "Not found"
        if len(parts) > 1:
            after_cal = parts[1].strip()
            # Grab the first "unit-like" string (numbers and letters)
            cal_match = re.search(r'^[:\s]*([\d\.\w\-x/]+)', after_cal)
            caliber = cal_match.group(1).strip() if cal_match else "Not found"

        return w_type, caliber

    # Apply to dataframe
    df[['type', 'caliber']] = df['description_text'].apply(lambda x: pd.Series(simple_extract(x)))

    return df[['model', 'type', 'caliber']]

# Run it
final_df = clean_weapon_data(text_df)
print(final_df.head(15))

                        model  \
0         38sws-dot38-caliber   
1                       ak-12   
2                       ak-47   
3       arctic-warfare-magnum   
4                  beretta-92   
5                   colt-1911   
6                   daewoo-k2   
7                desert-eagle   
8       double-barrel-shotgun   
9                   dp-27-lmg   
10                     fn-p90   
11                    fn-scar   
12                      glock   
13                  glock-18c   
14  glock-19-9mm-luger-pistol   

                                                                           type  \
0                                                                     Not found   
1                                                                 Assault rifle   
2                                                                 Assault rifle   
3                                                                     Not found   
4                                                       

# 3 Preprocessing

## 3.1 Audio Preprocessing

De preprocessing richt zich op het omzetten van de ruwe audio naar bruikbare features voor de pipeline. Hiervoor wordt RMS-energie berekend om de volumevariaties van het geluid stabiel te meten. Daarna worden pieken in de energiecurve gedetecteerd met drempels voor hoogte, prominente pieken en minimale afstand tussen pieken. Dit zorgt ervoor dat echo’s of achtergrondgeluid niet worden meegeteld. Dit is namelijk wat wij tijdens EDA hebben al onderzocht. Per firearm worden de resultaten van alle 25 bestanden gemiddeld om twee features te verkrijgen: avg_shots_per_sec (het gemiddelde aantal schoten per seconde) en avg_loudness (gemiddelde piekamplitude).

In [ ]:
from audio_dataset import extract_avg_features
firearm_features = extract_avg_features("data/audio/original_dataset/")
print(firearm_features.head(20))

## 3.2 Image Preprocessing

### Overzicht

Voor de afbeeldingsverwerking wordt een drie-staps pipeline gebruikt:

1. **Organisatie**: Afbeeldingen worden in mappenstructuur per wapen geplaatst
2. **Verwerking**: Achtergronden verwijderd en wapens gemeten (hoogte/breedte)
3. **Classificatie**: Onderscheid tussen één- en tweehandige wapens op basis van aspect ratio
4. **Analyse**: Visualisatie en export van resultaten

Het doel is om uit ruwe afbeeldingen gestructureerde features te extracten die bruikbaar zijn voor machine learning modellen.

### Stap 1: Organisatie van Afbeeldingen

**Waarom?**
- Afbeeldingen liggen momenteel los in `data/images/`
- De verwerkingsfunctie `process_weapon_images()` verwacht een structuur met submappen per wapen
- Dit maakt het mogelijk om meerdere afbeeldingen per wapen te verwerken

**Hoe?**
- Voor elke JPG/PNG-file wordt een submap gemaakt naar wapenaam
- Afbeeldingen worden gekopieerd naar `data/images/raw/[weapon_name]/`
- Dit zorgt voor schaalbare verwerking van grote datasets

**Output:**
- Georganiseerde mappenstructuur klaar voor batch-verwerking


In [ ]:
# Stap 1: Voorbereiding - Organiseer afbeeldingen in mappen per wapen
import shutil

# De afbeeldingen liggen nu rechtstreeks in data/images/
# We moeten ze verplaatsen naar data/images/raw/[weapon_name]/

raw_dir = "data/images/raw"
os.makedirs(raw_dir, exist_ok=True)

# Zet alle JPG-bestanden in submappen
for img_file in os.listdir("data/images"):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Maak de wapenaam (verwijder extensie)
        weapon_name = os.path.splitext(img_file)[0]
        weapon_dir = os.path.join(raw_dir, weapon_name)
        os.makedirs(weapon_dir, exist_ok=True)
        
        # Verplaats afbeelding
        src = os.path.join("data/images", img_file)
        dst = os.path.join(weapon_dir, img_file)
        
        if os.path.exists(src) and not os.path.exists(dst):
            shutil.copy(src, dst)
            print(f"✓ Gekopieerd: {img_file} → {weapon_dir}/")

print("\n✓ Afbeeldingen zijn georganiseerd in submappen!")


### Stap 2: Achtergrondverwijdering & Wapenmeting

**Waarom?**
- Achtergronden leiden tot ruis in visuele features (kleur, textuur)
- Rembg-model verwijdert achtergrond met AI (U²-Net segmentatie)
- Na verwijdering kan het wapen exact worden gemeten in pixels

**Techniek:**
1. **Rembg (Background Removal):** Deep learning model voor semantische segmentatie
   - Input: Originele afbeelding
   - Output: PNG met alpha-kanaal (transparante achtergrond)

2. **Contourdetectie (OpenCV):** Vind de wapenvorm
   - Analyseert alpha-kanaal voor silhouet
   - Bepaalt bounding box van grootste contour
   - Berekent pixel-dimensies: breedte (w) en hoogte (h)

3. **Aspect Ratio:** w/h ratio bepaalt wapentype
   - Ratio > 2.0: Langwerpig → waarschijnlijk geweer met kolf → 2-handed
   - Ratio ≤ 2.0: Compacter → waarschijnlijk pistool → 1-handed

**Output:**
- `processed_image_path`: Pad naar PNG zonder achtergrond
- `pixel_width`, `pixel_height`: Gemeten afmetingen
- `aspect_ratio`: Hoogte/breedte verhouding
- `handling_type`: Classificatie (1-handed/2-handed)
- `stock_label_nl`: Nederlandse beschrijving


In [ ]:
# Stap 2: Voer beeldverwerking uit (achtergrond verwijderen + meten)

# Verwerk alle wapens
processed_df = process_weapon_images(
    raw_base_dir="data/images/raw",
    processed_base_dir="data/images/processed"
)

# Opslaan naar CSV
processed_df.to_csv("data/images/weapon_analysis.csv", index=False)

print("\n" + "="*60)
print("RESULTATEN BEELDVERWERKING")
print("="*60)
print(f"✓ Total processed: {len(processed_df)} images")
print(f"✓ Saved to: data/images/weapon_analysis.csv")
print("\n" + processed_df.to_string())


### Stap 3: Visualisatie & Analyse

**Waarom?**
- Exploratieve Data Analysis (EDA) onthult patronen en distributie
- Visuele validatie of de classificatie correct werkt
- Detecteert anomalieën en uitbijters

**Visualisaties:**

1. **Handling Type Distribution:** 
   - Bar chart telt hoe veel wapens als 1-handed vs 2-handed geclassificeerd zijn
   - Vertaalt abstract naar concrete aantallen

2. **Aspect Ratio Histogram:**
   - Toont verdeling van w/h verhoudingen
   - Rode lijn op 2.0 markeert classificatie-threshold
   - Links van lijn: pistolen (compacter), rechts: geweren (langer)

3. **Wapen Dimensies Scatter Plot:**
   - X-as: pixel-breedte, Y-as: pixel-hoogte
   - Kleur aangeduid door stock_prediction (rood=2-handed, blauw=1-handed)
   - Visualiseert fysieke afmetingen per wapen

4. **Verdeling Pie Chart:**
   - Procentuele verdeling van handling types
   - Toont balans in dataset

**Samenvatting per Wapen:**
- Gemiddelde afmetingen en aspect ratio per wapentype
- Razendsnel overzicht van alle verwerkte wapens


In [ ]:
# Stap 3: Visualisatie & Analyse

# Distributieverloop
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Handling type verdeling (1-handed vs 2-handed)
handling_counts = processed_df['handling_type'].value_counts()
axes[0, 0].bar(handling_counts.index, handling_counts.values, color=['skyblue', 'coral'])
axes[0, 0].set_title('Handling Type Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Count')
for i, v in enumerate(handling_counts.values):
    axes[0, 0].text(i, v + 0.1, str(v), ha='center', fontweight='bold')

# 2. Aspect Ratio distributie
axes[0, 1].hist(processed_df['aspect_ratio'], bins=20, color='green', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(2.0, color='red', linestyle='--', linewidth=2, label='Threshold (2.0)')
axes[0, 1].set_title('Aspect Ratio Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Aspect Ratio (Width/Height)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()

# 3. Pixel dimensions
axes[1, 0].scatter(processed_df['pixel_width'], processed_df['pixel_height'], 
                   c=processed_df['has_stock_prediction'], cmap='coolwarm', s=100, alpha=0.6)
axes[1, 0].set_title('Wapen Dimensies (Pixel)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Width (pixels)')
axes[1, 0].set_ylabel('Height (pixels)')

# 4. Aantal afbeeldingen per handling type
stock_dist = processed_df[['handling_type', 'has_stock_prediction']].groupby('handling_type').size()
axes[1, 1].pie(stock_dist.values, labels=[f"{x}\n(n={y})" for x, y in zip(stock_dist.index, stock_dist.values)],
               autopct='%1.1f%%', colors=['#ff9999', '#66b3ff'])
axes[1, 1].set_title('Verdeling 1-Handed vs 2-Handed', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Samenvatting per wapen
print("\n" + "="*80)
print("SAMENVATTING PER WAPEN")
print("="*80)
summary = processed_df.groupby('weapon_key').agg({
    'pixel_width': 'mean',
    'pixel_height': 'mean',
    'aspect_ratio': 'mean',
    'handling_type': 'first',
    'stock_label_nl': 'first'
}).round(2)
summary.columns = ['Avg Width', 'Avg Height', 'Avg Ratio', 'Handling', 'Stock Label']
print(summary.to_string())


### Stap 4: Database Opslag (Load Phase) idk if to keep this or nah

**Waarom?**
- **Persistentie:** Data blijft behouden na sessie
- **Queryability:** SQL-queries op grote datasets efficiënter dan Pandas
- **Integratie:** Andere tools kunnen rechtstreeks uit database lezen
- **ETL-pipeline voltooid:** Extract → Transform → Load

**Hoe?**
- SQLite database (`weapons_data.db`) voor lokale opslag
- Tabel `image_features` met alle verwerkte imagedata
- Eenvoudig: `to_sql()` converteert DataFrame naar SQL-tabel

**Output:**
- Relatie-database met structured wapen-afbeelding features
- Kan later samengevoegd worden met audio- en tekstdata


In [ ]:
import sqlite3

# Maak verbinding met je database
conn = sqlite3.connect("weapons_data.db")

# Sla de verwerkte image data op in de database
processed_df.to_sql('image_features', conn, if_exists='replace', index=False)

print("✅ Image data succesvol geladen in de SQL database!")

# Check of het erin staat
check_df = pd.read_sql_query("SELECT weapon_key, handling_type FROM image_features LIMIT 5", conn)
print(check_df)

conn.close()

## 3.3 Text Preprocessing

In [ ]:
print(audio_df.head(50))